In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class NN(nn.Module):
    def __init__(self, inputs=4, h1=8, h2=9, outputs=3):
        super().__init__()
        self.fc1 = nn.Linear(inputs,h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, outputs)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

In [3]:
model = NN()
model

NN(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [4]:
df = pd.read_csv('iris.csv')
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [5]:
X = df.drop('target',axis=1).values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.parameters

<bound method Module.parameters of NN(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)>

In [7]:
epochs = 5
losses = []

for i in range(epochs):
    i+=1
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)

    print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  1  loss: 1.17306709
epoch:  2  loss: 1.13020027
epoch:  3  loss: 1.09773386
epoch:  4  loss: 1.07122600
epoch:  5  loss: 1.05541337


In [8]:
with torch.no_grad():
    y_val = model.forward(X_test)
    loss = criterion(y_val, y_test)
print(f'{loss:.8f}')

1.00540423


In [9]:
correct = 0
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_val = model.forward(data)
        print(f'{i+1:2}. {str(y_val):38}  {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1. tensor([0.2119, 0.2857, 0.5737])        1
 2. tensor([0.2199, 0.3125, 0.6041])        1
 3. tensor([0.3685, 0.1032, 0.4397])        0
 4. tensor([0.2014, 0.3171, 0.6002])        1
 5. tensor([0.2040, 0.3892, 0.6445])        2
 6. tensor([0.1754, 0.4585, 0.6393])        2
 7. tensor([0.3781, 0.1444, 0.4754])        0
 8. tensor([0.3981, 0.1007, 0.4313])        0
 9. tensor([0.1967, 0.4020, 0.6419])        2
10. tensor([0.1884, 0.4356, 0.6476])        2
11. tensor([0.1883, 0.4512, 0.6562])        2
12. tensor([0.3570, 0.0651, 0.4072])        0
13. tensor([0.1772, 0.4259, 0.6241])        2
14. tensor([0.2013, 0.3223, 0.6031])        1
15. tensor([0.1857, 0.3859, 0.6140])        2
16. tensor([0.2124, 0.2931, 0.5917])        1
17. tensor([0.1898, 0.3425, 0.5953])        2
18. tensor([0.4019, 0.1095, 0.4396])        0
19. tensor([0.2175, 0.3466, 0.6102])        1
20. tensor([0.2064, 0.4474, 0.6855])        2
21. tensor([0.3712, 0.0887, 0.4283])        0
22. tensor([0.4215, 0.1440, 0.4672